In [13]:
import ee
import zipfile
import geopandas as gpd
import ee
import pandas as pd

ee.Authenticate()
ee.Initialize(project='ee-caiosimplicioarantes')


In [14]:
shapefile_path = "./shape/contorno_area_total.shp"

# Load shapefile (supports both .shp and .zip formats)
if shapefile_path.endswith('.zip'):
    aoi = gpd.read_file(f'zip://{shapefile_path}')
else:
    aoi = gpd.read_file(shapefile_path)

# Dissolve geometries if multiple exist
aoi = aoi.dissolve()

geometry = aoi.geometry.iloc[0]

# Convert to GeoJSON and remove third dimension
geojson = geometry.__geo_interface__
geojson['coordinates'] = [
    [coord[:2] for coord in ring] if geojson['type'] == 'Polygon' else
    [[coord[:2] for coord in ring] for ring in polygon]
    for ring in geojson['coordinates']
]

# Create Earth Engine FeatureCollection
aoi = ee.FeatureCollection([ee.Feature(ee.Geometry(geojson))])


In [15]:
# Define the start and end dates for filtering the image collection

import pandas as pd
from datetime import datetime
inicio = '2024-11-11'
final = '2025-03-01'
nuvem = 30

# Load the Sentinel-2 image collection and filter by date, location, and cloud coverage
sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate(inicio, final) \
    .filterBounds(aoi) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', nuvem)) \
    .map(lambda image: image.set('date', image.date().format('YYYY-MM-dd')))

# Get the number of images in the collection
count = sentinel2.size().getInfo()
print(f"Number of images in collection: {count}")

Number of images in collection: 22


In [16]:
import os
import tempfile
import datetime
import requests
import concurrent.futures
from functools import partial

def download_all_sentinel_rgb_images(sentinel2, aoi, max_workers=4):
    """
    Baixa todas as imagens RGB da coleção Sentinel-2 como PNGs usando processamento paralelo.
    
    Args:
        sentinel2: Coleção de imagens Sentinel-2
        aoi: Área de interesse
        max_workers (int): Número máximo de downloads paralelos
    
    Returns:
        list: Lista de caminhos dos arquivos PNG baixados
    """
    temp_dir = tempfile.gettempdir()
    os.makedirs(temp_dir, exist_ok=True)
    
    # Pre-compute aoi bounds to avoid repeated calculations
    aoi_bounds = aoi.geometry().bounds().getInfo() if aoi else None
    
    # Get image list once
    image_list = sentinel2.toList(sentinel2.size())
    collection_size = image_list.size().getInfo()
    
    print(f"Baixando {collection_size} imagens Sentinel-2 em paralelo...")
    
    # Create a partial function with fixed arguments
    download_func = partial(
        _download_single_image, 
        image_list=image_list,
        aoi=aoi,
        temp_dir=temp_dir,
        aoi_bounds=aoi_bounds
    )
    
    # Use ThreadPoolExecutor for parallel downloads
    downloaded_files = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all download tasks
        future_to_idx = {executor.submit(download_func, i): i for i in range(collection_size)}
        
        # Process results as they complete
        for future in concurrent.futures.as_completed(future_to_idx):
            idx = future_to_idx[future]
            try:
                result = future.result()
                if result:
                    downloaded_files.append(result)
                    print(f"Imagem {idx+1}/{collection_size} baixada: {result}")
            except Exception as e:
                print(f"Erro ao baixar imagem {idx+1}/{collection_size}: {str(e)}")
    
    print(f"Download concluído. {len(downloaded_files)} imagens salvas.")
    return downloaded_files

def _download_single_image(i, image_list, aoi, temp_dir, aoi_bounds):
    """Helper function to download a single image by index"""
    # Get the image
    image = ee.Image(image_list.get(i))
    
    # Get date for filename
    try:
        image_date = image.date().format('YYYY-MM-dd').getInfo()
    except:
        image_date = f"image_{i}"
    
    # Clip to area of interest
    image = image.clip(aoi)
    
    # Create RGB visualization
    rgb_viz = image.visualize(bands=['B4', 'B3', 'B2'], min=200, max=2300)
    
    # Create unique filename
    timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S%f")
    file_name = f"sentinel2_rgb_{image_date}_{timestamp}.png"
    output_path = os.path.join(temp_dir, file_name)
    
    # Download the image
    try:
        url = rgb_viz.getDownloadUrl({
            'scale': 10,  # 10m resolution for Sentinel-2 RGB bands
            'format': 'png',
            'region': aoi_bounds
        })
        
        response = requests.get(url, timeout=60)
        response.raise_for_status()
        
        with open(output_path, 'wb') as f:
            f.write(response.content)  # Using content instead of streaming for smaller files
        
        return output_path
    except Exception as e:
        print(f"Error downloading image index {i}: {str(e)}")
        return None
    
image_paths = download_all_sentinel_rgb_images(sentinel2, aoi)



Baixando 22 imagens Sentinel-2 em paralelo...
Imagem 2/22 baixada: C:\Users\pc\AppData\Local\Temp\sentinel2_rgb_2024-11-11_20250625194212636459.png
Imagem 1/22 baixada: C:\Users\pc\AppData\Local\Temp\sentinel2_rgb_2024-11-11_20250625194212993607.png
Imagem 5/22 baixada: C:\Users\pc\AppData\Local\Temp\sentinel2_rgb_2024-11-26_20250625194215297424.png
Imagem 4/22 baixada: C:\Users\pc\AppData\Local\Temp\sentinel2_rgb_2024-11-19_20250625194214078773.png
Imagem 3/22 baixada: C:\Users\pc\AppData\Local\Temp\sentinel2_rgb_2024-11-19_20250625194212236061.png
Imagem 6/22 baixada: C:\Users\pc\AppData\Local\Temp\sentinel2_rgb_2024-11-26_20250625194216111970.png
Imagem 7/22 baixada: C:\Users\pc\AppData\Local\Temp\sentinel2_rgb_2024-12-19_20250625194218026321.png
Imagem 8/22 baixada: C:\Users\pc\AppData\Local\Temp\sentinel2_rgb_2024-12-19_20250625194218113469.png
Imagem 10/22 baixada: C:\Users\pc\AppData\Local\Temp\sentinel2_rgb_2024-12-31_20250625194220347437.png
Imagem 11/22 baixada: C:\Users\pc\A

In [17]:
sentinel2
aoi = aoi

def calculate_timeseries():
    """Calculates the time series of the selected vegetation index for the
    AOI."""
    """Calcula a série temporal do índice de vegetação selecionado para a
    AOI."""
    vegetation_index = "NDVI"

    # Define the vegetation index calculation in a function
    def calculate_index(image):
        if vegetation_index == "NDVI":
            index_image = image.normalizedDifference(["B8", "B4"]).rename("index")
        elif vegetation_index == "EVI":
            index_image = image.expression(
                "2.5 * ((NIR / 10000 - RED / 10000) / (NIR / 10000 + 6 * RED / 10000 - 7.5 * BLUE / 10000 + 1))",
                {
                    "NIR": image.select("B8"),
                    "RED": image.select("B4"),
                    "BLUE": image.select("B2"),
                },
            ).rename("index")
        elif vegetation_index == "SAVI":
            L = 0.5
            index_image = image.expression(
                "(1 + L) * ((NIR / 10000) - (RED / 10000)) / ((NIR / 10000) + (RED / 10000) + L)",
                {
                    "NIR": image.select("B8"),
                    "RED": image.select("B4"),
                    "L": L,
                },
            ).rename("index")
        elif vegetation_index == "GCI":
            index_image = image.expression(
                "NIR / GREEN - 1",
                {"NIR": image.select("B8"), "GREEN": image.select("B3")},
            ).rename("index")
        elif vegetation_index == "GNDVI":
            index_image = image.normalizedDifference(["B8", "B3"]).rename("index")
        else:
            raise ValueError(f"Unsupported vegetation index: {vegetation_index}")

        # Calculate mean value for the index over AOI
        mean_index = (
            index_image.reduceRegion(
                reducer=ee.Reducer.mean(), geometry=aoi, scale=10, bestEffort=True
            )
            .get("index")
        )

        return image.set({"mean_index": mean_index})

    # Map the calculation function over the collection and get results
    result = sentinel2.map(calculate_index)
    result = result.filter(ee.Filter.notNull(["mean_index"]))

    # Retrieve dates and mean index values separately using aggregate_array
    dates = result.aggregate_array("date").getInfo()
    mean_indices = result.aggregate_array("mean_index").getInfo()
    image_ids = result.aggregate_array("system:index").getInfo()

    # Combine the dates, mean indices, and image IDs into a DataFrame
    df = pd.DataFrame(
        {"date": dates, "AOI_average": mean_indices, "image_id": image_ids}
    )
    return df

In [18]:
df = calculate_timeseries()
df

,date,AOI_average,image_id
0,2024-11-11,0.163325,20241111T131241_20241111T131240_T22KHV
1,2024-11-11,0.163809,20241111T131241_20241111T131240_T23KKQ
2,2024-11-19,0.184058,20241119T132229_20241119T132231_T22KHV
3,2024-11-19,0.184087,20241119T132229_20241119T132231_T23KKQ
4,2024-11-26,0.212067,20241126T131249_20241126T131243_T22KHV
5,2024-11-26,0.212841,20241126T131249_20241126T131243_T23KKQ
6,2024-12-19,0.560628,20241219T132229_20241219T132232_T22KHV
7,2024-12-19,0.559907,20241219T132229_20241219T132232_T23KKQ
8,2024-12-31,0.847769,20241231T131241_20241231T131242_T22KHV
9,2024-12-31,0.846767,20241231T131241_20241231T131242_T23KKQ


In [19]:
df['image_paths'] = image_paths
df

df['image_paths'] = df['image_paths'].str.replace('\\', '/')

In [ ]:
import dash
from dash import dcc, html
import plotly.graph_objects as go
from dash.dependencies import Input, Output
import pandas as pd
import os
import base64
import numpy as np
from datetime import datetime

def create_sentinel_visualization_app(df):
    """
    Cria um aplicativo Dash para visualizar valores médios de AOI e imagens Sentinel-2.
    
    Args:
        df: DataFrame com colunas 'date', 'AOI_average', e 'image_paths'
    
    Returns:
        app: Aplicativo Dash
    """
    # Inicializar aplicativo Dash
    app = dash.Dash(__name__)
    
    # Criar gráfico de linha
    fig = go.Figure(go.Scatter(
        x=df['date'], 
        y=df['AOI_average'], 
        mode="lines+markers",
        marker=dict(size=10),
        hoverinfo="text",
        text=[f"Data: {date}<br>Valor médio: {val:.2f}" for date, val in zip(df['date'], df['AOI_average'])]
    ))
    
    fig.update_layout(
        title="Valores Médios de AOI ao Longo do Tempo",
        xaxis_title="Data",
        yaxis_title="Valor Médio na Área de Interesse",
        hovermode="closest"
    )
    
    app.layout = html.Div([
        # Layout principal com flexbox horizontal
        html.Div([
            # Container esquerdo para imagem e informações
            html.Div([
                # Imagem e informações em layout vertical
                html.Div([
                    # Container da imagem
                    html.Img(id="hover-image", src="", style={
                        "width": "400px",
                        "display": "none",
                        "max-height": "400px",
                        "object-fit": "contain",
                        "border": "1px solid #ddd"
                    }),
                    
                    # Informações do ponto abaixo da imagem
                    html.P(id="point-data", children="", style={
                        "text-align": "center",
                        "font-size": "14px",
                        "margin-top": "10px",
                        "display": "none",
                        "font-family": "monospace"
                    })
                ], style={
                    "display": "flex",
                    "flex-direction": "column",
                    "align-items": "center",
                    "justify-content": "center"
                })
            ], style={
                "width": "420px",
                "display": "flex",
                "align-items": "center",
                "justify-content": "center",
                "margin-right": "20px",
                "padding": "10px"
            }),

            # Gráfico de linha (lado direito)
            html.Div([
                dcc.Graph(id="line-plot", figure=fig)
            ], style={"flex": "1"})
        ], style={
            "display": "flex",
            "flex-direction": "row",
            "align-items": "center",
            "justify-content": "flex-start",
            "width": "100%",
            "height": "100%"
        })
    ])
    
    # Callback para atualizar imagem e informações ao passar o mouse
    @app.callback(
        [Output("hover-image", "src"),
         Output("hover-image", "style"),
         Output("point-data", "children"),
         Output("point-data", "style")],
        Input("line-plot", "hoverData")
    )
    def display_hover_data(hoverData):
        if hoverData:
            try:
                # Obter índice do ponto
                point_idx = hoverData["points"][0]["pointIndex"]
                
                # Obter dados deste ponto
                date = df['date'].iloc[point_idx]
                aoi_value = df['AOI_average'].iloc[point_idx]
                img_path = df['image_paths'].iloc[point_idx]  # Fixed column name here
                
                # Verificar se a imagem existe e codificar em base64
                if os.path.exists(img_path):
                    with open(img_path, 'rb') as f:
                        img_data = f.read()
                        encoded_img = base64.b64encode(img_data).decode('ascii')
                        img_src = f'data:image/png;base64,{encoded_img}'
                    
                    # Formatar informações do ponto
                    point_info = f"Data: {date}\nValor médio: {aoi_value:.4f}"
                    
                    img_style = {
                        "width": "400px",
                        "display": "block",
                        "max-height": "400px",
                        "object-fit": "contain",
                        "border": "1px solid #ddd"
                    }
                    info_style = {
                        "text-align": "center",
                        "font-size": "14px",
                        "margin-top": "10px",
                        "display": "block",
                        "white-space": "pre-line",
                        "font-family": "monospace"
                    }
                    return img_src, img_style, point_info, info_style
                else:
                    # Image doesn't exist
                    return "", {"display": "none"}, f"Image not found: {img_path}", {
                        "text-align": "center",
                        "font-size": "14px",
                        "margin-top": "10px",
                        "display": "block",
                        "color": "red",
                        "font-family": "monospace"
                    }
            except Exception as e:
                # Handle any errors gracefully
                return "", {"display": "none"}, f"Error: {str(e)}", {
                    "text-align": "center", 
                    "font-size": "14px",
                    "margin-top": "10px",
                    "display": "block",
                    "color": "red",
                    "font-family": "monospace"
                }
        
        # Retorno padrão quando não há hover
        return "", {"display": "none"}, "", {"display": "none"}
    
    return app


app = create_sentinel_visualization_app(df)
app.run_server(debug=True)

OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run_server.

In [ ]:
print(df.image_paths[0])

C:/Users/pc/AppData/Local/Temp/sentinel2_rgb_2024-11-11_20250307114424072296.png


In [ ]:
df.drop(columns=['image_id'], inplace=True)
df

,date,AOI_average,image_paths
0,2024-11-11,0.163325,C:/Users/pc/AppData/Local/Temp/sentinel2_rgb_2...
1,2024-11-11,0.163809,C:/Users/pc/AppData/Local/Temp/sentinel2_rgb_2...
2,2024-11-19,0.184058,C:/Users/pc/AppData/Local/Temp/sentinel2_rgb_2...
3,2024-11-19,0.184087,C:/Users/pc/AppData/Local/Temp/sentinel2_rgb_2...
4,2024-11-26,0.212067,C:/Users/pc/AppData/Local/Temp/sentinel2_rgb_2...
5,2024-11-26,0.212841,C:/Users/pc/AppData/Local/Temp/sentinel2_rgb_2...
6,2024-12-19,0.560628,C:/Users/pc/AppData/Local/Temp/sentinel2_rgb_2...
7,2024-12-19,0.559907,C:/Users/pc/AppData/Local/Temp/sentinel2_rgb_2...
8,2024-12-31,0.847769,C:/Users/pc/AppData/Local/Temp/sentinel2_rgb_2...
9,2024-12-31,0.846767,C:/Users/pc/AppData/Local/Temp/sentinel2_rgb_2...


In [ ]:
df.to_csv('df.csv', index=False)